Abrindo pacotes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

import xgboost as xgb

Abrindo dados

In [2]:
dic = pd.read_parquet("hdfs:///projetos/covariate_shift/bureau/book_positivo_dicionary.parquet", engine="pyarrow", use_threads=True) 
dic.shape

(14446, 4)

In [3]:
with open("data/importance_names.txt", "rb") as fp:   # Unpickling
    importance_names = pickle.load(fp)

In [4]:
def levenshtein(seq1, seq2):
    seq1, seq2 = seq1.lower(), seq2.lower()
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix[x, 0] = x
    for y in range(size_y):
        matrix[0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix[x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix[x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    #print (matrix)
    return (matrix[size_x - 1, size_y - 1])/max(len(seq1),len(seq2))

In [11]:
importance_names[:20]

['zqtcontratoativoa30d',
 'zqtfatura60dpgfullprz',
 'zqtcontratoativoa90d',
 'zpont_vl_ct',
 'zpont_qt_ct',
 'zvlparcaberto',
 'zvlparcabertou90d',
 'zpont_geral_qt',
 'zvlfaturaabertaa10drec',
 'zqtcontrgruposemgarantia',
 'zqtcontratoconsigativoa30d',
 'zminqtparcpgconsrec',
 'zmedfatura90dpgfullprz',
 'zmedfatura60dpgfullprz',
 'zmedqtparcpg',
 'zmaxmaxdatrasoparcnpg',
 'zmedfatura180dpgfullprz',
 'zvlfaturaabertarec',
 'zqtcontratoativoa12m',
 'zmaxmedfaturapgmais70perc']

In [5]:
%%time
m=50

ld=[np.argmin(np.array([levenshtein(v, var) for var in dic.loc[:,'Variavel'].values])) for v in importance_names[:m]]
#[np.min(np.array([levenshtein(v, var) for var in dic.loc[:,'Variavel'].values])) for v in list(X_t0)[:m]]

CPU times: user 6min 43s, sys: 5.03 s, total: 6min 48s
Wall time: 6min 38s


In [7]:
importance_names[:5]

['zqtcontratoativoa30d',
 'zqtfatura60dpgfullprz',
 'zqtcontratoativoa90d',
 'zpont_vl_ct',
 'zpont_qt_ct']

In [6]:
pd.set_option('display.max_colwidth', -1)
feat = pd.DataFrame({'var_dataset':importance_names[:m], 'var_dic':dic.iloc[ld,0],'book':dic.iloc[ld,2], 'descricao':dic.iloc[ld,3]})
feat

/opt/cloudera/parcels/Anaconda3-2018.12/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,var_dataset,var_dic,book,descricao
13574,zqtcontratoativoa30d,QtContratoAtivoA30d,Book Positivo,Quantidade de contratos de empréstimos e financiamentos ativos a mais de 30 dias
13442,zqtfatura60dpgfullprz,QtFatura60dPgPrz,Book Positivo,"Quantidade de faturas de cartão de crédito pagas nos últimos 60 dias, dentro do prazo"
13576,zqtcontratoativoa90d,QtContratoAtivoA90d,Book Positivo,Quantidade de contratos de empréstimos e financiamentos ativos a mais de 90 dias
14215,zpont_vl_ct,PERC_VL_CT,Book Positivo,Percentual do valor de parcelas e faturas pagas no prazo sobre a soma dos valores de todas as parcelas pagas de cartão de crédito
14207,zpont_qt_ct,PERC_QT_CT,Book Positivo,Percentual de parcelas e faturas pagas no prazo sobre todas as parcelas pagas de cartão de crédito
13940,zvlparcaberto,VlParcAberto,Book Positivo,"Soma os valores de parcelas de Empréstimos e Financiamentos nos últimos 12 meses, que ainda não foram pagas"
13946,zvlparcabertou90d,VlParcAbertoU90d,Book Positivo,"Soma os valores de parcelas de Empréstimos e Financiamentos em aberto (sem pagamento) em T0, atrasadas em até 90 dias"
14205,zpont_geral_qt,PERC_GERAL_QT,Book Positivo,"Percentual de parcelas e faturas pagas no prazo sobre todas as parcelas pagas de cartão de crédito, empréstimos e financiamentos"
13460,zvlfaturaabertaa10drec,QtFaturaAbertaA10d,Book Positivo,"Quantidade de faturas de cartão de crédito que estão em aberto (sem pagamento) em T0, com mais de 10 dias de atraso"
50,zqtcontrgruposemgarantia,QtContumacia,Book Cheques,"Quantidade de contumácia de cheques sustados, extraviados ou cancelados"
